In [2]:
# control random processes
import torch
torch.manual_seed(1)
import numpy as np
np.random.seed(1)

In [3]:
from network import NeuralNetwork

In [5]:
for duration in [.5, 1, 1.5, 2, 2.5, 3]:
    print(duration)
    network = NeuralNetwork()
    network.train(duration=duration, game_penalty_type='scores',
                 forget_after=40)

0.5
[[   8    1]
 [  16    3]
 [  32   28]
 [  64  264]
 [ 128  848]
 [ 256  771]
 [ 512   84]
 [1024    1]]
1
[[ 32  35]
 [ 64 337]
 [128 846]
 [256 698]
 [512  84]]
1.5
[[ 32  21]
 [ 64 186]
 [128 808]
 [256 857]
 [512 128]]
2
[[  8   3]
 [ 16 126]
 [ 32 556]
 [ 64 850]
 [128 425]
 [256  39]
 [512   1]]
2.5
[[ 32  30]
 [ 64 227]
 [128 866]
 [256 806]
 [512  71]]
3
[[  16    1]
 [  32   12]
 [  64  140]
 [ 128  582]
 [ 256 1101]
 [ 512  164]]


In [3]:
for duration in [1,2,3]:
    for learning_rate in [.001,.01]:
        network = NeuralNetwork()
        network.train(duration=duration, move_penalty_type='nonzero', forget_after=60,
                     game_penalty_type='scores', lr=learning_rate)

[[  32   25]
 [  64  180]
 [ 128  711]
 [ 256  846]
 [ 512  237]
 [1024    1]]
[[ 16   3]
 [ 32  33]
 [ 64 251]
 [128 751]
 [256 903]
 [512  59]]
[[  16    4]
 [  32   63]
 [  64  338]
 [ 128  801]
 [ 256  714]
 [ 512   78]
 [1024    2]]
[[  8   8]
 [ 16 105]
 [ 32 581]
 [ 64 967]
 [128 326]
 [256  13]]
[[  8   1]
 [ 16  30]
 [ 32 164]
 [ 64 517]
 [128 730]
 [256 510]
 [512  48]]
[[ 16   5]
 [ 32 102]
 [ 64 542]
 [128 801]
 [256 499]
 [512  51]]


In [4]:
for game_penalty in ['scores','log2_max']:
    for learning_rate in [0.001, 0.01, .1]:
        for forget_interval in [20,60,100]:
            print(game_penalty, learning_rate, forget_interval)
            network = NeuralNetwork()
            network.train(duration=1, move_penalty_type='nonzero', forget_after=forget_interval,
                     game_penalty_type=game_penalty, lr=learning_rate)

scores 0.001 20
[[  32   13]
 [  64  133]
 [ 128  626]
 [ 256 1085]
 [ 512  143]]
scores 0.001 60
[[  32    5]
 [  64  102]
 [ 128  658]
 [ 256  998]
 [ 512  234]
 [1024    3]]
scores 0.001 100
[[ 32   6]
 [ 64 190]
 [128 803]
 [256 903]
 [512  98]]
scores 0.01 20
[[ 16   1]
 [ 32  17]
 [ 64 157]
 [128 830]
 [256 895]
 [512 100]]
scores 0.01 60
[[ 32  21]
 [ 64 253]
 [128 782]
 [256 876]
 [512  68]]
scores 0.01 100
[[ 16   3]
 [ 32  42]
 [ 64 341]
 [128 994]
 [256 589]
 [512  31]]
scores 0.1 20
[[  8   4]
 [ 16  94]
 [ 32 489]
 [ 64 862]
 [128 518]
 [256  33]]
scores 0.1 60
[[  8   5]
 [ 16  93]
 [ 32 563]
 [ 64 815]
 [128 480]
 [256  44]]
scores 0.1 100
[[ 16   1]
 [ 32  34]
 [ 64 250]
 [128 781]
 [256 887]
 [512  47]]
log2_max 0.001 20
[[ 32  14]
 [ 64 202]
 [128 789]
 [256 869]
 [512 126]]
log2_max 0.001 60
[[ 16   1]
 [ 32  39]
 [ 64 230]
 [128 793]
 [256 885]
 [512  52]]
log2_max 0.001 100
[[  8   2]
 [ 16  35]
 [ 32 175]
 [ 64 473]
 [128 763]
 [256 485]
 [512  67]]
log2_max 0.01 

In [3]:
for game_penalty in ['scores','tile_sums']:
    for move_penalty in ['nonzero','linear_move_num']:
        for learning_rate in [0.001, 0.01, .1]:
            print(game_penalty, move_penalty, learning_rate)
            network = NeuralNetwork()
            network.train(duration=1, move_penalty_type=move_penalty,
                     game_penalty_type=game_penalty, lr=learning_rate)

scores nonzero 0.001
[[ 16   1]
 [ 32  10]
 [ 64 178]
 [128 801]
 [256 891]
 [512 119]]
scores nonzero 0.01
[[ 16   2]
 [ 32  47]
 [ 64 269]
 [128 774]
 [256 863]
 [512  45]]
scores nonzero 0.1
[[ 32  11]
 [ 64 170]
 [128 851]
 [256 876]
 [512  92]]
scores linear_move_num 0.001
[[ 16   3]
 [ 32  53]
 [ 64 310]
 [128 766]
 [256 799]
 [512  69]]
scores linear_move_num 0.01
[[ 16   4]
 [ 32  31]
 [ 64 239]
 [128 772]
 [256 892]
 [512  62]]
scores linear_move_num 0.1
[[ 32  21]
 [ 64 290]
 [128 872]
 [256 776]
 [512  41]]
log2_max nonzero 0.001
[[ 16   4]
 [ 32  42]
 [ 64 247]
 [128 778]
 [256 881]
 [512  48]]
log2_max nonzero 0.01
[[ 16   5]
 [ 32  32]
 [ 64 299]
 [128 853]
 [256 743]
 [512  68]]
log2_max nonzero 0.1


C:\Users\mpaige\Documents\2048\helper.py:72: RuntimeWarning: invalid value encountered in true_divide
  weights = weights/weights.sum()
C:\Users\mpaige\Documents\2048\helper.py:68: RuntimeWarning: invalid value encountered in less
  win_idx = np.random.choice(range(len(lst)), p=weights)


[[ 32  10]
 [ 64 175]
 [128 801]
 [256 916]
 [512  98]]
log2_max linear_move_num 0.001
[[  8   1]
 [ 16 108]
 [ 32 383]
 [ 64 712]
 [128 622]
 [256 168]
 [512   6]]
log2_max linear_move_num 0.01
[[ 16   3]
 [ 32  38]
 [ 64 240]
 [128 733]
 [256 926]
 [512  60]]
log2_max linear_move_num 0.1
[[  8   9]
 [ 16 121]
 [ 32 478]
 [ 64 829]
 [128 523]
 [256  40]]


In [3]:
for move_penalty in [None,'nonzero','linear_move_num','exponential_move_num']:
    for game_penalty in ['scores','max','log2_max','tile_sums']:
        network = NeuralNetwork()
        network.train(duration=1, move_penalty_type=move_penalty,
                     game_penalty_type=game_penalty)

[[ 16   2]
 [ 32  37]
 [ 64 282]
 [128 782]
 [256 766]
 [512 131]]
[[ 16   1]
 [ 32  23]
 [ 64 242]
 [128 776]
 [256 885]
 [512  73]]
[[ 32  14]
 [ 64 170]
 [128 803]
 [256 903]
 [512 110]]
[[  8   1]
 [ 16  17]
 [ 32 128]
 [ 64 571]
 [128 842]
 [256 405]
 [512  36]]
[[ 32  49]
 [ 64 269]
 [128 870]
 [256 747]
 [512  65]]
[[ 32  13]
 [ 64 171]
 [128 788]
 [256 937]
 [512  91]]
[[ 32  14]
 [ 64 190]
 [128 795]
 [256 895]
 [512 106]]
[[ 16   1]
 [ 32  52]
 [ 64 239]
 [128 634]
 [256 947]
 [512 127]]
[[ 16   1]
 [ 32   7]
 [ 64 183]
 [128 764]
 [256 967]
 [512  78]]
[[  8   4]
 [ 16 117]
 [ 32 540]
 [ 64 858]
 [128 439]
 [256  41]
 [512   1]]
[[ 32  18]
 [ 64 173]
 [128 739]
 [256 948]
 [512 122]]
[[ 32  30]
 [ 64 263]
 [128 789]
 [256 811]
 [512 107]]
[[ 16  20]
 [ 32 108]
 [ 64 409]
 [128 816]
 [256 598]
 [512  49]]
[[ 16   4]
 [ 32  40]
 [ 64 272]
 [128 763]
 [256 867]
 [512  54]]
[[ 32  34]
 [ 64 291]
 [128 791]
 [256 797]
 [512  87]]
[[  16    1]
 [  32   45]
 [  64  241]
 [ 128  634

In [ ]:
import pandas as pd
# mini-iterative, random_games=True, 5 hours
baseline = pd.DataFrame([[ 16,   0],
 [  32,   32],
 [  64,  351],
 [ 128, 1020],
 [ 256,  556],
 [ 512,   41]])



after = pd.DataFrame([[ 16,   1],
 [ 32,  20],
 [ 64, 236],
 [128, 758],
 [256, 910],
 [512,  75]])

import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2,1, sharex=True, sharey=True)

ax1.bar(baseline[0].astype(str), before[1])
ax1.set_ylabel('Before')

ax2.bar(after[0].astype(str), after[1])
ax2.set_ylabel('After')

fig.suptitle('Number of games reaching tile value,\nbefore and after training')

fig.show()

In [ ]:
params = {'random_games':[False, True],
         'random_frac':[None, np.random.uniform(0, 1)],
         'batch_size':10,
          'lr':0.001,
         'move_penalty_type':[None,'nonzero','linear_move_num','exponential_move_num'],
         'game_penalty_type':['scores','max','log2_max','tile_sums']}

In [ ]:
# test parameter combinations

for game_type in params['game_type']:
    for random_games in params['random_games']:
        for random_frac in params['random_frac']:
            for move_penalty_type in params['move_penalty_type']:
                for game_penalty_type in params['game_penalty_type']:
                    network.train(test=True, duration=1/60000,
                                 game_type=game_type, random_games=random_games, random_frac=random_frac,
                                 move_penalty_type=move_penalty_type, game_penalty_type=game_penalty_type)